In [1]:
import libraries as lib
import geopandas as gpd
import pandas as pd
import rasterio as rio
import xarray as xr
import rioxarray
!pip install openpyxl
import openpyxl

In [2]:
import os
from shapely.geometry import box
from pathlib import Path
import geopandas as gpd
import pandas as pd
import numpy as np
import xarray as xr
import xrspatial.multispectral as ms
from xrspatial.focal import mean, focal_stats, hotspots
import stackstac
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from collections import OrderedDict
from shapely.geometry import shape
import rioxarray
from rasterio.crs import CRS
from rasterio.plot import show
import rasterio.features
import itertools
import cartopy.crs as ccrs
import rioxarray
import dask.array as da
import joblib
import xarray as xr
from geopandas import GeoDataFrame
from pandas import Series
from xarray import Dataset
from flox.xarray import xarray_reduce
import warnings

In [3]:
Yasa_34kn = gpd.read_file("Yasa_34n_val.geojson").fillna(0)
Yasa_50kn = gpd.read_file("Yasa_50n_val.geojson").fillna(0)
Yasa_64kn = gpd.read_file("Yasa_64n_val.geojson").fillna(0)

In [4]:
print(Yasa_34kn.crs)
print(Yasa_50kn.crs)
print(Yasa_64kn.crs)

EPSG:4326
EPSG:4326
EPSG:4326


In [5]:
Yasa_34kn = Yasa_34kn.to_crs("EPSG:4326")
Yasa_50kn = Yasa_50kn.to_crs("EPSG:4326")
Yasa_64kn = Yasa_64kn.to_crs("EPSG:4326")

In [6]:
fiji_land = gpd.read_file("fiji-land.geojson").fillna(0).to_crs("EPSG:4326")

In [7]:
import geopandas as gpd
from shapely.geometry import mapping

# Clip the raster with the extracted geometries
land_Yasa_34kn = gpd.clip(fiji_land, Yasa_34kn)
land_Yasa_50kn = gpd.clip(fiji_land, Yasa_50kn)
land_Yasa_64kn = gpd.clip(fiji_land, Yasa_64kn)

In [10]:
land_Yasa_34kn = land_Yasa_34kn.to_crs("EPSG:3587")
land_Yasa_50kn = land_Yasa_50kn.to_crs("EPSG:3587")
land_Yasa_64kn = land_Yasa_64kn.to_crs("EPSG:3587")

In [11]:
# Save areas m2
land_Yasa_34kn['m2'] = land_Yasa_34kn.geometry.area
land_Yasa_50kn['m2'] = land_Yasa_50kn.geometry.area
land_Yasa_64kn['m2'] = land_Yasa_64kn.geometry.area

# Save areas km2
land_Yasa_34kn['km2'] = land_Yasa_34kn['m2']/1000000
land_Yasa_50kn['km2'] = land_Yasa_50kn['m2']/1000000
land_Yasa_64kn['km2'] = land_Yasa_64kn['m2']/1000000

In [12]:
Yasa_df = pd.concat([land_Yasa_34kn, land_Yasa_50kn, land_Yasa_64kn], axis=0)
# concatenated_df['Tropical Cyclone'] = ['TC Pam Vanuatu (2015)', 'TC Yasa Vanuatu (2020)', 'TC Winston Fiji (2016)', 'TC Yasa Fiji (2020)', 'TC Yasa Fiji (2021)']
Yasa_df['TC'] = "TC Yasa (2020)"
Yasa_df

,GID_0,COUNTRY,geometry,m2,km2,TC
0,FJI,Fiji,"MULTIPOLYGON (((-7946553.153 1204642.397, -794...",5.129747e+10,51297.474905,TC Yasa (2020)
0,FJI,Fiji,"MULTIPOLYGON (((-7744572.815 1262554.851, -774...",2.021983e+10,20219.830602,TC Yasa (2020)
0,FJI,Fiji,"MULTIPOLYGON (((-7642168.170 1121090.172, -764...",1.787221e+10,17872.205570,TC Yasa (2020)
